<a href="https://colab.research.google.com/github/EzraMW/ML/blob/main/NLP_HW_PMI%2C_Bayes%2C_Lift%2C_etc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from collections import Counter

# fetch 20 newsgroups dataset
#newsgroups_data = fetch_20newsgroups(subset='train')
cats = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_data = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),categories=cats)

Note that for now I only look at one category. As PMI is unsupervised, this is not a problem at all. I did this as the code already isn't super-fast to run.  Here is how GPT thought we should parse the corpus:

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## PMI

In [ ]:
docs = newsgroups_data['data']

# Preprocessing - convert docs to list of words
docs = [[word.lower() for word in sentence.split() if word not in stop_words] for sentence in docs]

# Create a word frequency dictionary
word_freq = Counter([word for doc in docs for word in doc])

# Filter words that appear less than 10 times
word_freq = {word: freq for word, freq in word_freq.items() if freq >= 30}

# Calculate number of docs
N = len(docs)

# Create a set of words to store words that have already been processed
processed_words = set()

This code has issues, but works. However, note that sometimes it gives a division by zero error, and weeds out terms that appear less than 10 times.  It also doesn't take out stopwords.  Now for the code. It has quite the loop within a loop using a dictionary (hash-table), but here it is: 

I have some questions:

1.   Do these terms help the classification task you started last week?
2.   Maybe other parameters work better?   Please check and justify different:
 , if taking out stopwords (or other 

*   Values for the frequency value (here 10), but 30 runs *much* faster
*   Only use those words with a PMI over a certain threshold that you experimentally tinker with. Note that this threshold is like confidence in the slides
*   Take out the stopwords first.

Now print those most correlated words.

What does this output mean to you?  I don't mean in a theoretical sense, but what can you do with it to help the classification performance?  We'll discuss in class.  Also, please output the most correlated words with the following list:

space, god, time, good, long, use, point, program, nasa, software, shuttle, moon, launch, data, orbit, lunar, graphics, software, image, earth, program, computer, christian

In [ ]:
words = ['space', 'god', 'time', 'good', 'long', 'use', 'point', 'program', 'nasa', 'software', 'shuttle', 'moon', 'launch', 'data', 'orbit', 'lunar', 'graphics', 'software', 'image', 'earth', 'program', 'computer', 'christian']

In [ ]:
#Calculate PMI for each word-word pair

pmi_dict = {}
for i, doc1 in enumerate(docs):
    for w1 in set(doc1) & set(words):
        if w1 in processed_words:
            continue
        processed_words.add(w1)
        p_w1 = word_freq[w1] / N
        for j, doc2 in enumerate(docs):
            for w2 in set(doc2) & set(word_freq.keys()):
                if w2 in processed_words or w1 == w2:
                    continue
                key = (w1, w2)
                if key in pmi_dict:
                    continue
                p_w2 = word_freq[w2] / N
                p_w1_w2 = len([d for d in docs if w1 in d and w2 in d]) / N
                pmi = np.log(p_w1_w2 / (p_w1 * p_w2))
                pmi_dict[key] = pmi
                pmi_dict[(w2, w1)] = pmi

<ipython-input-5-d335e83517b7>:19: RuntimeWarning: divide by zero encountered in log
  pmi = np.log(p_w1_w2 / (p_w1 * p_w2))


In [ ]:
num_correlated_words = 10
for word in words:
  correlated_words = sorted(pmi_dict.keys(), key=lambda x: pmi_dict[x], reverse=True)
  correlated_words = [(w2, pmi_dict[(word, w2)]) for w1, w2 in correlated_words if w1 == word][:num_correlated_words]
  print(f"Most correlated words for '{word}': {correlated_words}")

Most correlated words for 'space': [('space.', 0.5620154199129188), ('orbiting', 0.3994964904151439), ('office', 0.37611604177108027), ('exploration', 0.3613447244507679), ('fuel', 0.33887186859870927), ('agency', 0.33887186859870927), ('other', 0.29631225417991336), ('systems,', 0.27433334746113797), ('release', 0.27433334746113797), ('u.s.', 0.23177373304234192)]
Most correlated words for 'god': [("god's", 0.9207253289420209), ('people,', 0.8987464222232454), ('bible,', 0.8381218004068108), ('tells', 0.7761441001309131), ('do.', 0.7570959051602186), ('lord', 0.716424865429291), ('evil', 0.7009206788933258), ('life.', 0.6980757267610944), ("one's", 0.6911070574450008), ('exist', 0.6890258912411765)]
Most correlated words for 'time': [('spend', 1.1422305625380738), ('systems,', 1.1104818642234935), ('knew', 1.1104818642234935), ('present', 1.0914336692527988), ('amount', 1.0845063778202328), ('difficult', 1.083082890035379), ('responsible', 1.0797102055567398), ('eventually', 1.0414889

# Univariate

To see the top overall features based on Mutual Information, Chi Squared, and Anova see my previous homework here: https://github.com/EzraMW/ML/blob/main/Ezra_Wildes_20_newsgroups.ipynb

## Mutual Information, Bayes and Lift for Each Category

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
cats = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),categories=cats)
vectorizer = CountVectorizer(stop_words=stop_words)
vectors = vectorizer.fit_transform(newsgroups_train.data)

Most important Words for entire data set using mutual information

In [ ]:
from sklearn.feature_selection import mutual_info_classif
mutual_info = mutual_info_classif(vectors, newsgroups_train.target)
mscores = zip(vectorizer.get_feature_names_out(), mutual_info)
print("Mutual Info Scores:")
ms = sorted(mscores, key=lambda x:x[1], reverse=True)
for i in range(20):
  print(ms[i])

Mutual Info Scores:
('space', 0.1173236239203111)
('god', 0.08240250725184703)
('graphics', 0.08049773882433058)
('nasa', 0.055489959154984406)
('jesus', 0.05333519728289461)
('bible', 0.05004515233088479)
('religion', 0.04976531130996275)
('orbit', 0.049184367136139946)
('people', 0.04861661784666319)
('thanks', 0.04463381475169975)
('christian', 0.044047732361940616)
('launch', 0.04277924010634955)
('program', 0.040848159552271314)
('christians', 0.03895386885667414)
('shuttle', 0.037867437032728296)
('moon', 0.03768502661354253)
('atheism', 0.03634392268281014)
('file', 0.03534148343611571)
('files', 0.03505449223777356)
('religious', 0.035003673669855496)


### Find the label for each category

In [ ]:
for i in range(20,30):
  print("Label: ", newsgroups_train.target[i])
  print(newsgroups_train.data[i][:100])
  print(" ")

Label:  2



If by that you mean anything on the GD approach, there was an article on
it in a recent Avation W
 
Label:  1

The 68070 is a variation of the 68010 that was done a few years ago by
the European partners of Mot
 
Label:  3
PSA 145:9  The LORD is good to all: and his  tender  mercies  are
over all his works.
 
Label:  3
There has been a lot of discussion about Tyre.  In sum, Ezekiel prophesied
that the place would be m
 
Label:  1
About a year ago I started work on a problem that appeared to
be very simple and turned out to be qu
 
Label:  1
I need the file format for cc:Mail file formats - it seems to be PCX-based,
but with a twist: only t
 
Label:  2

 
Label:  1
I've got a 386 20Hz computer which is under warranty and my Trident
8900C video card is starting to 
 
Label:  2





 
Label:  3

	Please do! And if you don't want to post it here, email to me
:-) I don't know how this discussion
 


In [ ]:
label = 0
for i in range(len(newsgroups_train.data)):
  if newsgroups_train.target[i] == label:
    print("Label: ", label)
    print(newsgroups_train.data[i])
    print(" ")
    label += 1
  if label == 5:
    break

Label:  0
I have a request for those who would like to see Charley Wingate
respond to the "Charley Challenges" (and judging from my e-mail, there
appear to be quite a few of you.)  

It is clear that Mr. Wingate intends to continue to post tangential or
unrelated articles while ingoring the Challenges themselves.  Between
the last two re-postings of the Challenges, I noted perhaps a dozen or
more posts by Mr. Wingate, none of which answered a single Challenge.  

It seems unmistakable to me that Mr. Wingate hopes that the questions
will just go away, and he is doing his level best to change the
subject.  Given that this seems a rather common net.theist tactic, I
would like to suggest that we impress upon him our desire for answers,
in the following manner:

1. Ignore any future articles by Mr. Wingate that do not address the
Challenges, until he answers them or explictly announces that he
refuses to do so.

--or--

2. If you must respond to one of his articles, include within it
someth

So 0 is Atheism, 1 is graphics, 2 is space, and 3 is Religion

### Convert Target for Label
So for each category convert all target values in that category to 0 and all others to 1 so that we can do a this versus all others mutual information classifier

In [ ]:
atheism_target = np.where(newsgroups_train.target == 0, 0, 1)
graphics_target = np.where(newsgroups_train.target == 1, 0, 1)
space_target = np.where(newsgroups_train.target == 2, 0, 1)
religion_target = np.where(newsgroups_train.target == 3, 0, 1)

### Most Important Words for Each Category based on Mutual Information

In [ ]:
mutual_info = mutual_info_classif(vectors, atheism_target)
mscores = zip(vectorizer.get_feature_names_out(), mutual_info)
print("Most Important Atheism Words")
atheism_words = sorted(mscores, key=lambda x:x[1], reverse=True)
for i in range(10):
  print(atheism_words[i])
atheism_just_words = []
atheism_mut_scores = []
for word in atheism_words[:10]:
  atheism_just_words.append(word[0])
  atheism_mut_scores.append(word[1])

Most Important Atheism Words
('atheism', 0.032849840393006896)
('atheists', 0.026529748119225833)
('space', 0.024419661344439438)
('god', 0.0236410999751186)
('atheist', 0.02350049683641196)
('islam', 0.019920077487457846)
('bobby', 0.01898624552666018)
('graphics', 0.018582636095275953)
('program', 0.01838553353480459)
('religion', 0.01834529868962823)


In [ ]:
mutual_info = mutual_info_classif(vectors, graphics_target)
mscores = zip(vectorizer.get_feature_names_out(), mutual_info)
print("Most Important Graphics Words")
graphics_words = sorted(mscores, key=lambda x:x[1], reverse=True)
for i in range(10):
  print(graphics_words[i])
graphics_just_words = []
graphics_mut_scores = []
for word in graphics_words[:10]:
  graphics_just_words.append(word[0])
  graphics_mut_scores.append(word[1])

Most Important Graphics Words
('graphics', 0.078735449451137)
('thanks', 0.039237866280565165)
('file', 0.034024513710758646)
('hi', 0.032785155129032184)
('people', 0.032419921330152784)
('3d', 0.031090902377104136)
('files', 0.030094343392118736)
('god', 0.029495102939665285)
('windows', 0.028150002661353403)
('image', 0.0278426860435506)


In [ ]:
mutual_info = mutual_info_classif(vectors, space_target)
mscores = zip(vectorizer.get_feature_names_out(), mutual_info)
print("Most Important Space Words")
space_words = sorted(mscores, key=lambda x:x[1], reverse=True)
for i in range(10):
  print(space_words[i])
space_just_words = []
space_mut_scores = []
for word in space_words[:10]:
  space_just_words.append(word[0])
  space_mut_scores.append(word[1])

Most Important Space Words
('space', 0.11239488227225254)
('nasa', 0.05229455496688149)
('orbit', 0.04765845331318783)
('launch', 0.04201092889324566)
('shuttle', 0.036978340064979326)
('moon', 0.03584582368605021)
('lunar', 0.03042444304814209)
('spacecraft', 0.027884100207970747)
('god', 0.0270956539791017)
('solar', 0.02543282974861123)


In [ ]:
mutual_info = mutual_info_classif(vectors, religion_target)
mscores = zip(vectorizer.get_feature_names_out(), mutual_info)
print("Most Important Religion Words")
religion_words = sorted(mscores, key=lambda x:x[1], reverse=True)
for i in range(10):
  print(religion_words[i])
religion_just_words = []
religion_mut_scores = []
for word in religion_words[:10]:
  religion_just_words.append(word[0])
  religion_mut_scores.append(word[1])

Most Important Religion Words
('jesus', 0.03598687286090761)
('god', 0.029977189756657953)
('christ', 0.029892728222849895)
('christian', 0.029817873723415027)
('christians', 0.025354571173305862)
('bible', 0.022509815021592213)
('space', 0.022131912757651445)
('children', 0.01782911470279662)
('fbi', 0.015931068787980902)
('koresh', 0.015705900825574822)


In [ ]:
sum(word_freq.values())

114977

### Pre-compute Word Frequencies and Total Number of Words for Each Category

In [ ]:
cats = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_data = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),categories=cats)
docs = newsgroups_data['data']
# Preprocessing - convert docs to list of words
docs = [[word.lower() for word in sentence.split()] for sentence in docs]
# Create a word frequency dictionary
word_freq = Counter([word for doc in docs for word in doc])
all_freq = {word: freq for word, freq in word_freq.items()}
tot_words = sum(all_freq.values())
print(tot_words)

389558


In [ ]:
ath = ['alt.atheism']
newsgroups_data = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),categories=ath)
docs = newsgroups_data['data']
# Preprocessing - convert docs to list of words
docs = [[word.lower() for word in sentence.split()] for sentence in docs]
# Create a word frequency dictionary
word_freq = Counter([word for doc in docs for word in doc])
# Filter words that appear less than 30 times
ath_freq = {word: freq for word, freq in word_freq.items()}
ath_words = sum(ath_freq.values())

In [ ]:
rel = ['talk.religion.misc']
newsgroups_data = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),categories=rel)
docs = newsgroups_data['data']
# Preprocessing - convert docs to list of words
docs = [[word.lower() for word in sentence.split()] for sentence in docs]
# Create a word frequency dictionary
word_freq = Counter([word for doc in docs for word in doc])
# Filter words that appear less than 30 times
rel_freq = {word: freq for word, freq in word_freq.items()}
rel_words = sum(rel_freq.values())

In [ ]:
graph = ['comp.graphics']
newsgroups_data = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),categories=graph)
docs = newsgroups_data['data']
# Preprocessing - convert docs to list of words
docs = [[word.lower() for word in sentence.split()] for sentence in docs]
# Create a word frequency dictionary
word_freq = Counter([word for doc in docs for word in doc])
# Filter words that appear less than 30 times
graph_freq = {word: freq for word, freq in word_freq.items()}
graph_words = sum(word_freq.values())

In [ ]:
spc = ['sci.space']
newsgroups_data = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),categories=spc)
docs = newsgroups_data['data']
# Preprocessing - convert docs to list of words
docs = [[word.lower() for word in sentence.split()] for sentence in docs]
# Create a word frequency dictionary
word_freq = Counter([word for doc in docs for word in doc])
# Filter words that appear less than 30 times
spc_freq = {word: freq for word, freq in word_freq.items()}
spc_words = sum(spc_freq.values())

### Atheism

In [ ]:
# helper function to deal with NoneType returns from frequency calls
def default_to_zero(val):
  if val == None:
    return 0
  else:
    return val

calculate the bayes and lift for this category

In [ ]:
bayes_list = []
lifts_list = []
for word in atheism_just_words:
  print("Word: ", word)
  # get the frequency of the word in this category
  word_frequency = default_to_zero(ath_freq.get(word))
  category_freq = word_frequency/ath_words

  # now we add up the frequency of this word from all the other categories
  other_frequency = default_to_zero(rel_freq.get(word)) + default_to_zero(graph_freq.get(word)) + default_to_zero(spc_freq.get(word))
  other_num_words = rel_words + graph_words + spc_words
  other_freq = other_frequency/other_num_words

  bayes = category_freq/other_freq
  bayes_list.append(bayes)
  print("Bayes: ", bayes)

  # to calculate the lift, we just add this category's frequency and num_words to the denominator
  # so that it calculates the category frequency over the total frequency
  other_frequency += default_to_zero(ath_freq.get(word))
  other_num_words += ath_words
  tot_freq = other_frequency/other_num_words

  lift = category_freq/tot_freq
  lifts_list.append(lift)
  print("Lift: ", lift)
  print(" ")

Word:  atheism
Bayes:  238.13273672364824
Lift:  4.120167588469056
 
Word:  atheists
Bayes:  103.9846283693264
Lift:  4.051396396128189
 
Word:  space
Bayes:  0.02483783433884206
Lift:  0.0324070102689416
 
Word:  god
Bayes:  4.168431518067586
Lift:  2.369925704671734
 
Word:  atheist
Bayes:  111.12861047103586
Lift:  4.059128068639884
 
Word:  islam
Bayes:  22.622609988746582
Lift:  3.6612443063072715
 
Word:  bobby
Bayes:  92.077991533144
Lift:  4.0359330511048
 
Word:  graphics
Bayes:  0.0
Lift:  0.0
 
Word:  program
Bayes:  0.07029748685568951
Lift:  0.09043544742921249
 
Word:  religion
Bayes:  8.527419905532547
Lift:  3.0423232301832495
 


In [ ]:
import pandas as pd
ath_df = pd.DataFrame(data=[atheism_just_words, atheism_mut_scores, bayes_list, lifts_list], index=["Words", "Mutual Information", "Bayes", "Lifts"]).T
ath_df

,Words,Mutual Information,Bayes,Lifts
0,atheism,0.03285,238.132737,4.120168
1,atheists,0.02653,103.984628,4.051396
2,space,0.02442,0.024838,0.032407
3,god,0.023641,4.168432,2.369926
4,atheist,0.0235,111.12861,4.059128
5,islam,0.01992,22.62261,3.661244
6,bobby,0.018986,92.077992,4.035933
7,graphics,0.018583,0.0,0.0
8,program,0.018386,0.070297,0.090435
9,religion,0.018345,8.52742,3.042323


### Religion

In [ ]:
bayes_list = []
lifts_list = []
for word in religion_just_words:
  print("Word: ", word)
  # get the frequency of the word in this category
  word_frequency = default_to_zero(rel_freq.get(word))
  category_freq = word_frequency/rel_words

  # now we add up the frequency of this word from all the other categories
  other_frequency = default_to_zero(ath_freq.get(word)) + default_to_zero(graph_freq.get(word)) + default_to_zero(spc_freq.get(word))
  other_num_words = ath_words + graph_words + spc_words
  other_freq = other_frequency/other_num_words

  bayes = category_freq/other_freq
  bayes_list.append(bayes)
  print("Bayes: ", bayes)

  # to calculate the lift, we just add this category's frequency and num_words to the denominator
  # so that it calculates the category frequency over the total frequency
  other_frequency += default_to_zero(rel_freq.get(word))
  other_num_words += rel_words
  tot_freq = other_frequency/other_num_words

  lift = category_freq/tot_freq
  lifts_list.append(lift)
  print("Lift: ", lift)
  print(" ")

rel_df = pd.DataFrame(data=[religion_just_words, religion_mut_scores, bayes_list, lifts_list], index=["Words", "Mutual Information", "Bayes", "Lifts"]).T
rel_df

Word:  jesus
Bayes:  6.507496278962735
Lift:  2.9725429935292063
 
Word:  god
Bayes:  2.6066684047205677
Lift:  1.93528281530835
 
Word:  christ
Bayes:  41.75943054152054
Lift:  4.260754443321643
 
Word:  christian
Bayes:  6.289852115725919
Lift:  2.9361550227493565
 
Word:  christians
Bayes:  6.536258693455389
Lift:  2.9772352476626387
 
Word:  bible
Bayes:  4.5726912670264035
Lift:  2.5813551509789
 
Word:  space
Bayes:  0.004703697965929324
Lift:  0.005991274035831097
 
Word:  children
Bayes:  11.983140938001545
Lift:  3.554218822791057
 
Word:  fbi
Bayes:  105.30639006122571
Lift:  4.476879668707524
 
Word:  koresh
Bayes:  18.51939963145693
Lift:  3.87203272646835
 


,Words,Mutual Information,Bayes,Lifts
0,jesus,0.035987,6.507496,2.972543
1,god,0.029977,2.606668,1.935283
2,christ,0.029893,41.759431,4.260754
3,christian,0.029818,6.289852,2.936155
4,christians,0.025355,6.536259,2.977235
5,bible,0.02251,4.572691,2.581355
6,space,0.022132,0.004704,0.005991
7,children,0.017829,11.983141,3.554219
8,fbi,0.015931,105.30639,4.47688
9,koresh,0.015706,18.5194,3.872033


### Graphics

In [ ]:
bayes_list = []
lifts_list = []
for word in graphics_just_words:
  print("Word: ", word)
  # get the frequency of the word in this category
  word_frequency = default_to_zero(graph_freq.get(word))
  category_freq = word_frequency/graph_words

  # now we add up the frequency of this word from all the other categories
  other_frequency = default_to_zero(ath_freq.get(word)) + default_to_zero(rel_freq.get(word)) + default_to_zero(spc_freq.get(word))
  other_num_words = ath_words + rel_words + spc_words
  other_freq = other_frequency/other_num_words

  bayes = category_freq/other_freq
  bayes_list.append(bayes)
  print("Bayes: ", bayes)

  # to calculate the lift, we just add this category's frequency and num_words to the denominator
  # so that it calculates the category frequency over the total frequency
  other_frequency += default_to_zero(graph_freq.get(word))
  other_num_words += graph_words
  tot_freq = other_frequency/other_num_words

  lift = category_freq/tot_freq
  lifts_list.append(lift)
  print("Lift: ", lift)
  print(" ")

graph_df = pd.DataFrame(data=[graphics_just_words, graphics_mut_scores, bayes_list, lifts_list], index=["Words", "Mutual Information", "Bayes", "Lifts"]).T
graph_df

Word:  graphics
Bayes:  354.9520957434111
Lift:  4.188757619310011
 
Word:  thanks
Bayes:  6.603759920807649
Lift:  2.8394021114101444
 
Word:  file
Bayes:  41.34385206102232
Lift:  3.920821868671299
 
Word:  hi
Bayes:  19.361023404186064
Lift:  3.6230033434551516
 
Word:  people
Bayes:  0.2737284439961053
Lift:  0.33051960326257523
 
Word:  3d
Bayes:  193.61023404186062
Lift:  4.157544820358371
 
Word:  files
Bayes:  14.687672927313564
Lift:  3.4654814589571017
 
Word:  god
Bayes:  0.0235535564527811
Lift:  0.030629255319065296
 
Word:  windows
Bayes:  72.60383776569773
Lift:  4.046971819816925
 
Word:  image
Bayes:  29.56683419088879
Lift:  3.810924188119492
 


,Words,Mutual Information,Bayes,Lifts
0,graphics,0.078735,354.952096,4.188758
1,thanks,0.039238,6.60376,2.839402
2,file,0.034025,41.343852,3.920822
3,hi,0.032785,19.361023,3.623003
4,people,0.03242,0.273728,0.33052
5,3d,0.031091,193.610234,4.157545
6,files,0.030094,14.687673,3.465481
7,god,0.029495,0.023554,0.030629
8,windows,0.02815,72.603838,4.046972
9,image,0.027843,29.566834,3.810924


### Space

In [ ]:
bayes_list = []
lifts_list = []
for word in space_just_words:
  print("Word: ", word)
  # get the frequency of the word in this category
  word_frequency = default_to_zero(spc_freq.get(word))
  category_freq = word_frequency/spc_words

  # now we add up the frequency of this word from all the other categories
  other_frequency = default_to_zero(ath_freq.get(word)) + default_to_zero(rel_freq.get(word)) + default_to_zero(graph_freq.get(word))
  other_num_words = ath_words + rel_words + graph_words
  other_freq = other_frequency/other_num_words
  
  # deal with situations where word has zero frequency in other categories
  if other_freq == 0:
    other_freq = .001

  bayes = category_freq/other_freq
  bayes_list.append(bayes)
  print("Bayes: ", bayes)

  # to calculate the lift, we just add this category's frequency and num_words to the denominator
  # so that it calculates the category frequency over the total frequency
  other_frequency += default_to_zero(spc_freq.get(word))
  other_num_words += spc_words
  tot_freq = other_frequency/other_num_words

  lift = category_freq/tot_freq
  lifts_list.append(lift)
  print("Lift: ", lift)
  print(" ")

spc_df = pd.DataFrame(data=[space_just_words, space_mut_scores, bayes_list, lifts_list], index=["Words", "Mutual Information", "Bayes", "Lifts"]).T
spc_df

Word:  space
Bayes:  43.4615775216328
Lift:  3.087373797958662
 
Word:  nasa
Bayes:  34.30409501979579
Lift:  3.0473828905612166
 
Word:  orbit
Bayes:  61.79231089810377
Lift:  3.133063540284343
 
Word:  launch
Bayes:  128.6403563242342
Lift:  3.1912507523473685
 
Word:  shuttle
Bayes:  84.82399041466972
Lift:  3.163199752636638
 
Word:  moon
Bayes:  69.65678683058971
Lift:  3.1455245884559284
 
Word:  lunar
Bayes:  1.4086267972494269
Lift:  3.24699312356741
 
Word:  spacecraft
Bayes:  0.758491352365076
Lift:  3.24699312356741
 
Word:  god
Bayes:  0.016401409661075984
Lift:  0.02352893567802471
 
Word:  solar
Bayes:  215.71133986247133
Lift:  3.2135189676543434
 


,Words,Mutual Information,Bayes,Lifts
0,space,0.112395,43.461578,3.087374
1,nasa,0.052295,34.304095,3.047383
2,orbit,0.047658,61.792311,3.133064
3,launch,0.042011,128.640356,3.191251
4,shuttle,0.036978,84.82399,3.1632
5,moon,0.035846,69.656787,3.145525
6,lunar,0.030424,1.408627,3.246993
7,spacecraft,0.027884,0.758491,3.246993
8,god,0.027096,0.016401,0.023529
9,solar,0.025433,215.71134,3.213519


# 2-Gram PMI

In [3]:
cats = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_data = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),categories=cats)

In [4]:
docs = newsgroups_data['data']

# Preprocessing - convert docs to list of words
docs = [[word.lower() for word in sentence.split() if word not in stop_words] for sentence in docs]

# Create a word frequency dictionary
word_freq = Counter([word for doc in docs for word in doc])

# Filter words that appear less than 10 times
word_freq = {word: freq for word, freq in word_freq.items() if freq >= 30}

# Calculate number of docs
N = len(docs)

# Create a set of words to store words that have already been processed
processed_words = set()

Now we want to use PMI to calculate the most correlated words to the following given word list

In [ ]:
words = ['space', 'god', 'time', 'good', 'long', 'use', 'point', 'program', 'nasa', 'software', 'shuttle', 'moon', 'launch', 'data', 'orbit', 'lunar', 'graphics', 'software', 'image', 'earth', 'program', 'computer', 'christian']

In [ ]:
#Calculate PMI for each word-word pair

pmi_dict = {}
for i, doc1 in enumerate(docs):
    for w1 in set(doc1) & set(words):
        if w1 in processed_words:
            continue
        processed_words.add(w1)
        p_w1 = word_freq[w1] / N
        for j, doc2 in enumerate(docs):
            for w2 in set(doc2) & set(word_freq.keys()):
                if w2 in processed_words or w1 == w2:
                    continue
                key = (w1, w2)
                if key in pmi_dict:
                    continue
                p_w2 = word_freq[w2] / N
                p_w1_w2 = len([d for d in docs if w1 in d and w2 in d]) / N
                pmi = np.log(p_w1_w2 / (p_w1 * p_w2))
                pmi_dict[key] = pmi
                pmi_dict[(w2, w1)] = pmi

<ipython-input-34-d335e83517b7>:19: RuntimeWarning: divide by zero encountered in log
  pmi = np.log(p_w1_w2 / (p_w1 * p_w2))


In [ ]:
num_correlated_words = 10
for word in words:
  correlated_words = sorted(pmi_dict.keys(), key=lambda x: pmi_dict[x], reverse=True)
  correlated_words = [(w2, pmi_dict[(word, w2)]) for w1, w2 in correlated_words if w1 == word][:num_correlated_words]
  print(f"Most correlated words for '{word}': {correlated_words}")

Most correlated words for 'space': [('space.', 0.5620154199129188), ('orbiting', 0.3994964904151439), ('office', 0.37611604177108027), ('exploration', 0.3613447244507679), ('fuel', 0.33887186859870927), ('agency', 0.33887186859870927), ('other', 0.29631225417991336), ('systems,', 0.27433334746113797), ('release', 0.27433334746113797), ('u.s.', 0.23177373304234192)]
Most correlated words for 'god': [("god's", 0.9207253289420209), ('people,', 0.8987464222232454), ('bible,', 0.8381218004068108), ('tells', 0.7761441001309131), ('do.', 0.7570959051602186), ('lord', 0.716424865429291), ('evil', 0.7009206788933258), ('life.', 0.6980757267610944), ("one's", 0.6911070574450008), ('exist', 0.6890258912411765)]
Most correlated words for 'time': [('spend', 1.1422305625380738), ('systems,', 1.1104818642234935), ('knew', 1.1104818642234935), ('present', 1.0914336692527988), ('amount', 1.0845063778202328), ('difficult', 1.083082890035379), ('responsible', 1.0797102055567398), ('eventually', 1.0414889

# Performance

Again, for the performance (measured by f1 score) graphed relative to the number of features, see my previous homework: see my previous homework here: https://github.com/EzraMW/ML/blob/main/Ezra_Wildes_20_newsgroups.ipynb. 

See my comments there that stemming, removing capital letters didn't help the performance of the Multinomial Bayes model. Also, neither did stripping accent charachters or making the data binary. 

Lastly, using a Random Forrest model made it perform significantly worse.